# vercel/ai

> 

In [ ]:
from lovely_docs.settings import settings, GitSource
from lovely_docs.git import clone_repo
from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import logging

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [ ]:
source =  GitSource(
    name="vercel/ai",
    doc_dir="content/docs",
    repo="https://github.com/vercel/ai",
    commit="main",
    ecosystems=["js"])


commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

64: 3850.0/3850.0 completed with 306 local objects.

#### Read the markdown files

In [ ]:
tree = build_markdown_doc_tree(clone_dir/source.doc_dir, extensions=['.mdx'], exclude=["08-migration-guides"])
tree

DocItem(origPath=Path('.'), name='', displayName='', digest='', short_digest='', essence='', relevant=True, usage=Usage(input=0, output=0, details=None), token_counts=TokenCounts(fulltext=0, digest=0, short_digest=0), fulltext='', children=[DocItem(origPath=Path('00-introduction/index.mdx'), name='index.mdx', displayName='index.mdx', digest='', short_digest='', essence='', relevant=True, usage=Usage(input=0, output=0, details=None), token_counts=TokenCounts(fulltext=0, digest=0, short_digest=0), fulltext='---\ntitle: AI SDK by Vercel\ndescription: The AI SDK is the TypeScript toolkit for building AI applications and agents with React, Next.js, Vue, Svelte, Node.js, and more.\n---\n\n# AI SDK\n\nThe AI SDK is the TypeScript toolkit designed to help developers build AI-powered applications and agents with React, Next.js, Vue, Svelte, Node.js, and more.\n\n## Why use the AI SDK?\n\nIntegrating large language models (LLMs) into applications is complicated and heavily dependent on the speci

In [ ]:
assert tree
tree.children

[DocItem(origPath=Path('00-introduction/index.mdx'), name='index.mdx', displayName='index.mdx', digest='', short_digest='', essence='', relevant=True, usage=Usage(input=0, output=0, details=None), token_counts=TokenCounts(fulltext=0, digest=0, short_digest=0), fulltext='---\ntitle: AI SDK by Vercel\ndescription: The AI SDK is the TypeScript toolkit for building AI applications and agents with React, Next.js, Vue, Svelte, Node.js, and more.\n---\n\n# AI SDK\n\nThe AI SDK is the TypeScript toolkit designed to help developers build AI-powered applications and agents with React, Next.js, Vue, Svelte, Node.js, and more.\n\n## Why use the AI SDK?\n\nIntegrating large language models (LLMs) into applications is complicated and heavily dependent on the specific model provider you use.\n\nThe AI SDK standardizes integrating artificial intelligence (AI) models across [supported providers](/docs/foundations/providers-and-models). This enables developers to focus on building great AI applications,

#### Process the tree

In [ ]:
processed_tree = await process_tree_depth_first(settings, tree, source.name)

+3.429s DEBUG lovely_docs/docs.py:157 llm_process_page Processing 02-foundations/01-overview.mdx
+0.298s DEBUG lovely_docs/docs.py:157 llm_process_page Processing 02-foundations/02-providers-and-models.mdx
+0.019s DEBUG lovely_docs/docs.py:157 llm_process_page Processing 02-foundations/03-prompts.mdx
+0.019s DEBUG lovely_docs/docs.py:157 llm_process_page Processing 02-foundations/04-tools.mdx
+0.019s DEBUG lovely_docs/docs.py:157 llm_process_page Processing 02-foundations/05-streaming.mdx
+0.020s DEBUG lovely_docs/docs.py:157 llm_process_page Processing 02-foundations/index.mdx
+0.021s DEBUG lovely_docs/docs.py:157 llm_process_page Processing 02-getting-started/01-navigating-the-library.mdx
+0.019s DEBUG lovely_docs/docs.py:157 llm_process_page Processing 02-getting-started/02-nextjs-app-router.mdx
+0.017s DEBUG lovely_docs/docs.py:157 llm_process_page Processing 02-getting-started/03-nextjs-pages-router.mdx
+0.016s DEBUG lovely_docs/docs.py:157 llm_process_page Processing 02-getting-s

#### Calculate cost

In [ ]:
usage = calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 839,200
  Output tokens: 253,439
  Total tokens: 1,092,639
Cost:
  Total: $2.11
  Input: $0.84
  Output: $1.27


#### save the results

In [ ]:
save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)